# Ejecutar Ejecutables y Comandos con Privilegios

Esta demo complementa `demo_02_subprocess.ipynb` con ejemplos avanzados de:
- Ejecutar ejecutables específicos del sistema
- Ejecutar comandos con privilegios elevados (sudo)

---

## 1. Ejecutar Ejecutables Específicos

Puedes ejecutar cualquier ejecutable del sistema, ya sea binarios compilados o scripts.

In [1]:
import subprocess

# 1. Ejecutar Python como ejecutable
print("🐍 Ejecutar Python:")
try:
    resultado = subprocess.run(['python3', '--version'],
                              capture_output=True,
                              text=True)
    print(f"   {resultado.stdout.strip()}")
except FileNotFoundError:
    print("   ❌ python3 no disponible")

# 2. Ejecutar git
print("\n📦 Ejecutar Git:")
try:
    resultado = subprocess.run(['git', '--version'],
                              capture_output=True,
                              text=True)
    print(f"   {resultado.stdout.strip()}")
except FileNotFoundError:
    print("   ❌ git no disponible")

# 3. Ejecutar un script de bash (si existe)
print("\n📜 Ejecutar script bash:")
try:
    # Crear un script temporal
    with open('/tmp/test_script.sh', 'w') as f:
        f.write('#!/bin/bash\necho "Script ejecutado correctamente"\necho "Argumentos: $@"')
    
    # Dar permisos de ejecución
    subprocess.run(['chmod', '+x', '/tmp/test_script.sh'], check=True)
    
    # Ejecutar el script con argumentos
    resultado = subprocess.run(['/tmp/test_script.sh', 'arg1', 'arg2'],
                              capture_output=True,
                              text=True)
    print(f"   Salida:\n{resultado.stdout}")
    
    # Limpiar
    subprocess.run(['rm', '/tmp/test_script.sh'])
except Exception as e:
    print(f"   ❌ Error: {e}")

🐍 Ejecutar Python:
   Python 3.12.3

📦 Ejecutar Git:
   git version 2.43.0

📜 Ejecutar script bash:
   Salida:
Script ejecutado correctamente
Argumentos: arg1 arg2



In [2]:
# 4. Ejecutar comandos del sistema con rutas absolutas
print("📍 Ejecutar con ruta absoluta:")
import shutil

# Buscar la ruta completa del ejecutable
ruta_ls = shutil.which('ls')
if ruta_ls:
    print(f"   Ruta de 'ls': {ruta_ls}")
    resultado = subprocess.run([ruta_ls, '-lh', '/tmp'],
                              capture_output=True,
                              text=True)
    print(f"   Primeras 3 líneas de /tmp:")
    for linea in resultado.stdout.split('\n')[:3]:
        if linea:
            print(f"      {linea}")
else:
    print("   ❌ 'ls' no encontrado")

📍 Ejecutar con ruta absoluta:
   Ruta de 'ls': /usr/bin/ls
   Primeras 3 líneas de /tmp:
      total 88K
      -rw------- 1 user user    0 oct 20 16:33 config-err-Oq6x0J
      drwxrwxrwx 2 user user 4,0K oct 20 16:33 mintUpdate


## 2. Ejecutar Comandos con `sudo` (Privilegios Elevados)

### ⚠️ Consideraciones importantes:

1. **Requiere autenticación**: El usuario debe tener permisos en `/etc/sudoers`
2. **Entrada interactiva**: Por defecto, `sudo` pide contraseña
3. **Riesgos de seguridad**: Ejecutar con privilegios elevados puede ser peligroso

### Opciones para manejar `sudo`:

| Método | Descripción | Riesgo |
|--------|-------------|--------|
| **`-S`** | Lee password desde stdin | ⚠️ Medio (password en código) |
| **`-n`** | No pedir password (requiere configuración previa) | ✅ Bajo (si se configura bien) |
| **Configurar sudoers** | Permitir comandos sin password | ⚠️ Alto (si se configura mal) |
| **pkexec** | Alternativa gráfica a sudo | ✅ Medio |

### 🔐 Mejores prácticas:

- ✅ Usa `sudo -v` para verificar permisos antes de ejecutar
- ✅ Limita los comandos permitidos sin password en sudoers
- ❌ **NUNCA** pongas passwords en código fuente
- ✅ Considera alternativas como servicios con permisos específicos

In [3]:
# Método 1: sudo con validación previa (recomendado)
print("🔐 Método 1: Verificar permisos sudo")
try:
    # Verificar si tenemos permisos sudo (sin ejecutar nada peligroso)
    resultado = subprocess.run(['sudo', '-n', 'true'],
                              capture_output=True,
                              text=True,
                              timeout=2)
    
    if resultado.returncode == 0:
        print("   ✅ Usuario tiene permisos sudo sin password")
        
        # Ahora sí, ejecutar comando con privilegios
        resultado = subprocess.run(['sudo', 'whoami'],
                                  capture_output=True,
                                  text=True)
        print(f"   Ejecutando como: {resultado.stdout.strip()}")
        
    else:
        print("   ℹ️ Se requiere password para sudo")
        print("   (En entorno de producción, configura sudoers para comandos específicos)")

except subprocess.TimeoutExpired:
    print("   ⏱️ Timeout - probablemente pidiendo contraseña")
except FileNotFoundError:
    print("   ❌ sudo no disponible")
except Exception as e:
    print(f"   ❌ Error: {e}")

🔐 Método 1: Verificar permisos sudo
   ℹ️ Se requiere password para sudo
   (En entorno de producción, configura sudoers para comandos específicos)


In [4]:
# Método 2: Ejemplo de lectura de archivo de sistema (requiere sudo)
print("\n📂 Método 2: Leer archivo protegido")
try:
    # Intentar leer sin sudo (fallará en muchos sistemas)
    print("   Sin sudo:")
    resultado = subprocess.run(['cat', '/etc/sudoers'],
                              capture_output=True,
                              text=True)
    
    if resultado.returncode != 0:
        print(f"   ❌ Acceso denegado: {resultado.stderr.strip()}")
        
        # Ahora con sudo (requiere configuración previa o password)
        print("\n   Con sudo (requiere permisos):")
        resultado = subprocess.run(['sudo', '-n', 'cat', '/etc/sudoers'],
                                  capture_output=True,
                                  text=True,
                                  timeout=2)
        
        if resultado.returncode == 0:
            print("   ✅ Archivo leído (mostrando primeras 5 líneas):")
            for i, linea in enumerate(resultado.stdout.split('\n')[:5]):
                if linea and not linea.startswith('#'):
                    print(f"      {linea}")
        else:
            print("   ℹ️ Se necesita configurar acceso sin password")
    else:
        print("   ✅ Acceso permitido (raro, sistema muy permisivo)")
        
except subprocess.TimeoutExpired:
    print("   ⏱️ Timeout - esperando contraseña")
except Exception as e:
    print(f"   ❌ Error: {e}")


📂 Método 2: Leer archivo protegido
   Sin sudo:
   ❌ Acceso denegado: cat: /etc/sudoers: Permiso denegado

   Con sudo (requiere permisos):
   ℹ️ Se necesita configurar acceso sin password


In [5]:
# Método 3: Ejecutar script Python con sudo
print("\n🐍 Método 3: Script Python con privilegios elevados")
try:
    # Crear script temporal que necesita privilegios
    script_content = """#!/usr/bin/env python3
import os
print(f"UID actual: {os.getuid()}")
print(f"Usuario: {os.environ.get('USER', 'desconocido')}")
if os.getuid() == 0:
    print("✅ Ejecutando como root")
else:
    print("ℹ️ Ejecutando como usuario normal")
"""
    
    with open('/tmp/test_sudo.py', 'w') as f:
        f.write(script_content)
    
    # Dar permisos
    subprocess.run(['chmod', '+x', '/tmp/test_sudo.py'], check=True)
    
    # Ejecutar sin sudo
    print("   Sin sudo:")
    resultado = subprocess.run(['python3', '/tmp/test_sudo.py'],
                              capture_output=True,
                              text=True)
    print(resultado.stdout)
    
    # Ejecutar con sudo (si está configurado)
    print("   Con sudo (si está disponible):")
    resultado = subprocess.run(['sudo', '-n', 'python3', '/tmp/test_sudo.py'],
                              capture_output=True,
                              text=True,
                              timeout=2)
    
    if resultado.returncode == 0:
        print(resultado.stdout)
    else:
        print("   ℹ️ sudo requiere password o no está configurado")
    
    # Limpiar
    subprocess.run(['rm', '/tmp/test_sudo.py'])
    
except subprocess.TimeoutExpired:
    print("   ⏱️ Timeout - esperando contraseña interactiva")
    subprocess.run(['rm', '-f', '/tmp/test_sudo.py'])
except Exception as e:
    print(f"   ❌ Error: {e}")


🐍 Método 3: Script Python con privilegios elevados
   Sin sudo:
   Sin sudo:
UID actual: 1000
Usuario: user
ℹ️ Ejecutando como usuario normal

   Con sudo (si está disponible):
UID actual: 1000
Usuario: user
ℹ️ Ejecutando como usuario normal

   Con sudo (si está disponible):
   ℹ️ sudo requiere password o no está configurado
   ℹ️ sudo requiere password o no está configurado


### 💡 Configurar sudo sin password (para desarrollo)

Para ejecutar comandos específicos sin password, edita `/etc/sudoers`:

```bash
# Método seguro: usar visudo
sudo visudo

# Añadir línea (reemplaza 'usuario' y ajusta el comando):
usuario ALL=(ALL) NOPASSWD: /usr/bin/python3 /ruta/al/script.py
```

**Ejemplo más específico:**
```bash
# Permitir solo comandos específicos sin password
juan ALL=(ALL) NOPASSWD: /usr/bin/apt update
maria ALL=(ALL) NOPASSWD: /usr/bin/systemctl restart nginx
```

**⚠️ Advertencias:**
- Solo para entornos de desarrollo/testing controlados
- En producción, usa servicios con permisos específicos
- Nunca uses `NOPASSWD: ALL` (inseguro)

In [6]:
# Método 4: Verificar si un comando está disponible antes de ejecutarlo
print("🔍 Método 4: Verificación previa de comandos")
import shutil

comandos_admin = [
    'systemctl',    # Gestión de servicios
    'apt',          # Gestor de paquetes (Debian/Ubuntu)
    'yum',          # Gestor de paquetes (RedHat/CentOS)
    'journalctl',   # Logs del sistema
]

print("   Comandos administrativos disponibles:")
for cmd in comandos_admin:
    ruta = shutil.which(cmd)
    if ruta:
        # Verificar si podemos ejecutarlo con sudo
        try:
            resultado = subprocess.run(['sudo', '-n', cmd, '--version'],
                                      capture_output=True,
                                      text=True,
                                      timeout=1)
            
            if resultado.returncode == 0:
                print(f"   ✅ {cmd:15} → {ruta} (sudo OK)")
            else:
                print(f"   ⚠️  {cmd:15} → {ruta} (sudo requiere password)")
        except subprocess.TimeoutExpired:
            print(f"   ⏱️  {cmd:15} → {ruta} (timeout)")
        except:
            print(f"   ℹ️  {cmd:15} → {ruta}")
    else:
        print(f"   ❌ {cmd:15} → no encontrado")

🔍 Método 4: Verificación previa de comandos
   Comandos administrativos disponibles:
   ⚠️  systemctl       → /usr/bin/systemctl (sudo requiere password)
   ⚠️  apt             → /usr/local/bin/apt (sudo requiere password)
   ❌ yum             → no encontrado
   ⚠️  journalctl      → /usr/bin/journalctl (sudo requiere password)


## 📚 Resumen

### 🔧 Ejecutables:
- Usa `subprocess.run([ejecutable, args])` para ejecutar binarios
- Verifica disponibilidad con `shutil.which(comando)`
- Scripts bash: dar permisos con `chmod +x` antes de ejecutar

### 🔐 Sudo:
- **Método 1**: Verificar permisos con `sudo -n true`
- **Método 2**: Leer archivos protegidos
- **Método 3**: Ejecutar scripts con privilegios
- **Método 4**: Verificar comandos administrativos

### ⚠️ Seguridad:
- Solo usar sudo cuando sea absolutamente necesario
- Configurar sudoers de forma específica (no `NOPASSWD: ALL`)
- En producción, usar servicios con permisos específicos
- Nunca incluir passwords en el código

---

**Ver también:** `demo_02_subprocess.ipynb` para comandos básicos de subprocess.